In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-assignment-1/hindi_corpus.txt


In [6]:
devdf = pd.read_csv('../input/ud-hindi-treebank-sentence-mapping/UDdev.csv')
testdf = pd.read_csv('../input/ud-hindi-treebank-sentence-mapping/UDtest.csv')
traindf = pd.read_csv('../input/ud-hindi-treebank-sentence-mapping/UDtrain.csv')

In [7]:
devdf.head()

,Index,Output,Input
0,0,रामायण काल में भगवान राम का पुत्र कुश का राजधा...,रामायण काल में भगवान राम के पुत्र कुश की राजधा...
1,1,मल्ला का राजधानी हो के कारण प्राचीनकाल में यह ...,मल्‍लों की राजधानी होने के कारण प्राचीनकाल में...
2,2,बौद्ध धर्मावलंबी के अनुसार लुंबनी बोधगया और सा...,बौद्ध धर्मावलंबियों के अनुसार लुंबनी बोधगया और...
3,3,हिंदू राजा का काल में चीन से ह्वेन सांग फाह्या...,हिंदू राजाओं के काल में चीन से ह्वेन सांग फाह्...
4,4,कुशीनगर का सबसे ज्यादा महत्व बौद्ध तीर्थ के रू...,कुशीनगर का सबसे ज्‍यादा महत्‍व बौद्ध तीर्थ के ...


In [8]:
testdf.head()

,Index,Output,Input
0,0,यह अतिरिक्त गुग्गुल कुंड भीम गुफा तथा भीमशिला ...,इसके अतिरिक्त गुग्गुल कुंड भीम गुफा तथा भीमशिल...
1,1,आधा किमी का दूरी पर भैरवनाथ मंदिर है जहाँ केवल...,आधा किमी की दूरी पर भैरवनाथ मंदिर है जहाँ केवल...
2,2,भैरव का स्थान उत्तराखंड में क्षेत्रपाल अथवा भू...,भैरव का स्थान उत्तराखंड में क्षेत्रपाल अथवा भू...
3,3,यह सोनप्रयाग से किमी आगे और केदारनाथ में किमी ...,यह सोनप्रयाग से किमी आगे और केदारनाथ में किमी ...
4,4,यह ऊँचाई केवल मीटर है,इसकी ऊँचाई केवल मीटर है


In [9]:
traindf.head()

,Index,Output,Input
0,0,यह एशिया का सबसे बड़ा मस्जिद में से एक है,यह एशिया की सबसे बड़ी मस्जिदों में से एक है
1,1,यह नवाब शाहजेहन ने बनवा था,इसे नवाब शाहजेहन ने बनवाया था
2,2,यह प्रवेश द्वार दो मंजिला है,इसका प्रवेश द्वार दो मंजिला है
3,3,जो चार मेहराब है और मुख्य प्रार्थना हॉल में जा...,जिसमें चार मेहराबें हैं और मुख्य प्रार्थना हॉल...
4,4,पूरा इमारत बेहद खूबसूरत है,पूरी इमारत बेहद खूबसूरत है


In [10]:
def findContext(df, context):
    
    for i in range(len(df)):
        data = df.iloc[i]
        lemmaSent = data['Output']
        wordSent = data['Input']
        
        lemmas = lemmaSent.split()
        words = wordSent.split()
        
        for i in range(1, len(words)-1):
            context[words[i]] = lemmas[i] + "_" + words[i-1] + "_" + words[i+1]
                    
        if(i % 1000 == 0):
            print("Till row " + str(i) + " processed.")
            
    return context

In [11]:
context = findContext(traindf, {})
context = findContext(testdf, context)
context = findContext(devdf, context)
print(len(context))

17451


In [13]:
wordList = []
lemmaList = []
contextList = []

for word, values in context.items():
    values = values.split('_')
    wordList.append(word)
    lemmaList.append(values[0])
    contextList.append(values[1] + " " + word + " " + values[2])

In [14]:
print(len(wordList))
print(len(lemmaList))
print(len(contextList))

17451
17451
17451


In [15]:
contextdf = pd.DataFrame()
contextdf['Index'] = range(1, len(wordList) + 1)
contextdf['Lemma'] = lemmaList
contextdf['Derived Word'] = wordList
contextdf['Context'] = contextList
contextdf.head()

,Index,Lemma,Derived Word,Context
0,1,एशिया,एशिया,पश्चिम एशिया में
1,2,कर,की,माँग की थी
2,3,सबसे,सबसे,को सबसे ज्यादा
3,4,बडा,बड़ी,पर बड़ी तादाद
4,5,मस्जिद,मस्जिदों,और मस्जिदों और


In [16]:
contextdf.to_csv('./ContextUD.csv')